# Run pyCIAM

In [2]:
import sys

sys.path.append("../")

In [3]:
from pyCIAM.run import execute_pyciam
from shared import (
    AUTHOR,
    CONTACT,
    DIR_SCRATCH,
    HISTORY,
    PATH_OUTPUTS,
    PATH_PARAMS,
    PATH_REFA,
    PATH_SLIIDERS,
    PATH_SLIIDERS_SEG,
    PATH_SLR_AR5_QUANTILES,
    PATH_SLR_AR6,
    PATH_SLR_SWEET,
    PATHS_SURGE_LOOKUP,
    QUANTILES,
    STORAGE_OPTIONS,
    start_dask_cluster,
)

In [4]:
AR6_ONLY = False

TMPPATH = DIR_SCRATCH / "pyCIAM_results_quantiles_prechunked.zarr"

# When running on larger/scalable dask cluster, may wish to specify number of workers
# Default is LocalCluster which will use the number of CPUs available on local machine
N_WORKERS = 100

SEG_CHUNKSIZE = 2

SEG_ADM_SUBSET = None

DESCRIPTION = "Projected coastal damages from pyCIAM, using quantiles of SLR scenarios."

In [5]:
if AR6_ONLY:
    all_slr_ds = [PATH_SLR_AR6]
    all_slr_name = ["ar6"]
else:
    all_slr_ds = [PATH_SLR_AR6, PATH_SLR_AR5_QUANTILES, PATH_SLR_SWEET]
    all_slr_name = ["ar6", "ar5", "sweet"]

In [ ]:
client, cluster = start_dask_cluster()
cluster.scale(N_WORKERS)
cluster

In [8]:
execute_pyciam(
    PATH_PARAMS,
    PATH_SLIIDERS,
    all_slr_ds,
    all_slr_name,
    PATH_REFA,
    econ_input_path_seg=PATH_SLIIDERS_SEG,
    output_path=PATH_OUTPUTS,
    seg_var_subset=SEG_ADM_SUBSET,
    surge_input_paths=PATHS_SURGE_LOOKUP,
    pyciam_seg_chunksize=SEG_CHUNKSIZE,
    tmp_output_path=TMPPATH,
    quantiles=QUANTILES,
    dask_client_func=lambda: client,
    storage_options=STORAGE_OPTIONS,
    extra_attrs={
        "author": AUTHOR,
        "contact": CONTACT,
        "description": DESCRIPTION,
        "history": HISTORY,
    },
)